In [1]:
import pandas as pd
import os
import re
import numpy as np
from bokeh.models import ColumnDataSource
from data_access import MARKS, get_race_data

In [2]:
import bokeh
bokeh.__version__

u'0.12.5'

In [3]:
AC_schedule = {"170527": [["USA", "FRA"], ["SWE", "JPN"], ["NZL", "FRA"], ["SWE", "GBR"], ["USA", "NZL"], ["JPN", "GBR"],],
     "170528": [["SWE", "FRA"], ["USA", "GBR"], ["NZL", "JPN"], ["USA", "SWE"], ["NZL", "GBR"], ["USA", "JPN"],],
     "170529": [["FRA", "GBR"], ["SWE", "NZL"], ["JPN", "FRA"]],
     "170530": [["SWE", "NZL"], ["USA", "FRA"], ["SWE", "GBR"]]}

In [4]:
def get_recording_times(date):
    
    # eventually will add functionality to check if the relevant files have been downloaded
    # right now just assume that they have indeed been downloaded
    
    file_names = os.listdir(os.getcwd()+"/{}/csv".format(date))
    s = set([])
    for i in l:
        s.add(re.search('^[0-9]+', i).group())
    return sorted(list(s))    

def get_competitors(date, race_number):
    return AC_schedule[date][race_number-1]
    

In [5]:
def get_race_data(date, race_number):
    """
    date = string of the form "YYMMDD"
    race_number = int, race number within the day
    
    returns: [marks, boat1, boat2], list of pandas dataframes
    """
    rec_times = get_recording_times(date)
    boats = get_competitors(date, race_number)
    filetime = rec_times[race_number - 1]
    
    marks = ["SL1", "SL2"]
    res = []
    
    script_dir = os.getcwd() 

    for val in boats + marks:
        rel_path = "/{}/csv/{}-NAV-{}.csv".format(date, filetime, val)
        path = script_dir + rel_path
        res.append(pd.read_csv(path))
    return res

In [6]:
dat = get_race_data("170530", 1)

NameError: global name 'l' is not defined

In [ ]:
import pandas as pd
import os
import re

AC_schedule = {"170527": [["USA", "FRA"], ["SWE", "JPN"], ["NZL", "FRA"], ["SWE", "GBR"], ["USA", "NZL"], ["JPN", "GBR"],],
     "170528": [["SWE", "FRA"], ["USA", "GBR"], ["NZL", "JPN"], ["USA", "SWE"], ["NZL", "GBR"], ["USA", "JPN"],],
     "170529": [["FRA", "GBR"], ["SWE", "NZL"], ["JPN", "FRA"]],
     "170530": [["SWE", "NZL"], ["USA", "FRA"], ["SWE", "GBR"]]}

def get_recording_times(date):

    # eventually will add functionality to check if the relevant files have been downloaded
    # right now just assume that they have indeed been downloaded

    file_names = os.listdir(os.getcwd()+"/{}/csv".format(date))
    s = set([])
    for i in file_names:
        s.add(re.search('^[0-9]+', i).group())
    return sorted(list(s))

def get_competitors(date, race_number):
    return AC_schedule[date][race_number-1]


def get_race_data(date, race_number):
    """
    date = string of the form "YYMMDD"
    race_number = int, race number within the day

    returns: [marks, boat1, boat2], list of pandas dataframes
    """
    rec_times = get_recording_times(date)
    boats = get_competitors(date, race_number)
    filetime = rec_times[race_number - 1]

    marks = ["SL1", "SL2"]
    res = []

    script_dir = os.getcwd()
    res = {}
    for val in boats + marks:
        rel_path = "/{}/csv/{}-NAV-{}.csv".format(date, filetime, val)
        path = script_dir + rel_path
        res[val] = pd.read_csv(path)

    event_path = script_dir + "/{}/csv/{}_events.csv".format(date,filetime)
    res["events"] = pd.read_csv(event_path)
    
    for key in res.keys():
        res[key]['Secs'] = res[key]['Secs'].apply(lambda x: round(x, 1))
    return res
MARKS = ['SL1', 'SL2', 'FL1', 'Fl2', 'M1', 'LG1', 'LG2', 'U1', 'U2']


In [ ]:

def get_start_time(data):
    # returns start time (the gun) in s from midnight
    events_frame = data['events']
    return events_frame.loc[events_frame['Event'] == 'RaceStarted']['Secs'][0]

def get_mark_list(data):
    # returns positions of the marks at start time
    mark_data = pd.concat([data[m] for m in MARKS])
    start_time = get_start_time(data)
    return mark_data.loc[abs(mark_data['Secs'] - start_time) < .5]

In [ ]:
dat = get_race_data("170530", 1)

In [ ]:
t = ColumnDataSource(get_mark_list(dat))

In [ ]:
np.linspace(5.0,5.5,retstep=.1)

In [ ]:
dat['events'].columns

In [ ]:
dat['events'].loc[dat['events']['Event'] == 'RaceStarted']['Secs']

In [ ]:
dat['SWE'].loc[dat['SWE']['Secs']  == 50890.2]

In [ ]:
pd.concat([dat['SL1'], dat['SL2']])